<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #9: Modelado de temas
`Fecha de entrega: Mayo 6, 2021. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [2]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [3]:
# Cargar datos
path = 'C:/Users/johnv/reviews_vidjew_es.csv'
data = pd.read_csv(path)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [4]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto 

data['pp'] = data['review_body'].apply(lambda texto: pre_procesado(texto))

data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,pp
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


###  `[10 pts]` Punto 2: Modelo de LDA

In [5]:
# Crear una represantación de los documentos en forma de diccionario
dictionary = Dictionary(data['pp'].values)

# Filtrar palabras muy frecuentes o infrecuentes
#dictionary.filter_extremes(no_below=5, no_above=0.5) 

# Corpus
corpus = [dictionary.doc2bow(text) for text in data['pp'].values]

# Entrenar el modelo
model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = 7, passes=10)


model.print_topics(num_words=10)

[(0,
  '0.017*"calidad" + 0.014*"bien" + 0.013*"precio" + 0.011*"parece" + 0.010*"cadena" + 0.009*"aunque" + 0.009*"colgante" + 0.008*"llegado" + 0.007*"bonitos" + 0.007*"esperaba"'),
 (1,
  '0.012*"bien" + 0.011*"calidad" + 0.010*"día" + 0.009*"bastante" + 0.007*"llegó" + 0.007*"producto" + 0.007*"sido" + 0.006*"regalo" + 0.006*"pulsera" + 0.005*"bonito"'),
 (2,
  '0.016*"juego" + 0.012*"producto" + 0.010*"bien" + 0.009*"si" + 0.006*"viene" + 0.005*"plata" + 0.005*"foto" + 0.005*"original" + 0.005*"regalo" + 0.005*"todas"'),
 (3,
  '0.010*"si" + 0.010*"bien" + 0.009*"juego" + 0.007*"caja" + 0.007*"producto" + 0.006*"llego" + 0.006*"así" + 0.005*"problema" + 0.005*"dos" + 0.005*"ok"'),
 (4,
  '0.023*"bien" + 0.013*"juego" + 0.010*"si" + 0.007*"bonito" + 0.006*"llegó" + 0.006*"mando" + 0.005*"producto" + 0.005*"funciona" + 0.005*"regalo" + 0.005*"perfecto"'),
 (5,
  '0.010*"si" + 0.009*"mas" + 0.009*"calidad" + 0.008*"buena" + 0.007*"mando" + 0.007*"bien" + 0.007*"precio" + 0.006*"jugar

###  `[25 pts]` Punto 3: Visualización de LDA

In [6]:
def get_doc_top_n(text_processed, n):
    d = dictionary.doc2bow(text_processed)
    topics = dict(model.get_document_topics(d))
    try:
        return topics[n]
    except:
        return None
    
for t in range(0,7):
    top_name = f"topic_{t}"
    data[top_name] = data['pp'].apply(lambda doc: get_doc_top_n(doc, t))
    
for t in range(0,7):
    print(f"*********************************** TOPIC {t} ***********************************")
    topic = f"topic_{t}"
    for i,articulo in enumerate(data.sort_values('topic_0', ascending=False)['review_body'].values[:5]):
        print(f"Artículo #{i}")
        print(articulo[:500])
        print()
    print()

*********************************** TOPIC 0 ***********************************
Artículo #0
La pulsera es de muy buena calidad, la verdad es que era algo reacio a adquirir este tipo de regalos a través de Amazon, pero está pulsera me ha demostrado lo equivocado que estaba... Al ser de acero inoxidable está brillante como el primer día, a mi novia aun no se le ha puesto negra ni nada extraño y eso que no se la quita para nada, la lleva puesta 24/7. De hecho recientemente, fuimos a una boda nocturna y la utilizo de accesorio para combinar su vestido, y se llevó más de una mirada. Volvien

Artículo #1
Lo que es la edición coleccionista es muy pobre por lo que costó de salida, mucho dinero para tan poca cosa. Un artbook y un steelbook no justifica el precio de salida. Por no hablar de aquellos que piensan venderlo por barbaridades. Ningún fan debería pagar más del precio original. De verdad que no lo vale y yo me considero muy fan de zelda pero esa la verdad. En cuanto al juego, el remake 

###  `[3 pts]` Preguntas

- ¿Qué conclusiones puede sacar de los resultados de los últimos tres talleres, usando diferentes modelos pero los mismos datos?

In [ ]:
Para este modelo no tuve tan claro el tema del entrenamiento, me falto la ultima parte del modelo, ya que me arrojaba un error 
y no logre encontrar el motivo, sin embargo, me atreveria a decir que cada modelo es importante y que la decision de cual es el 
mejor va habitualmente en el que demore menos corriendo y este fue los que mas demoró en mi caso. Es importante revisar con 
otros set de datos si continua demorando mas que los anteriores y si el error que me arrojo con este set se debia al contenido
de los datos o falla durante la ejecución, me gustaria probarlo con otro set para descartar opciones.